In [1]:
# 결과 요약
#### Softmax----------------: 0.8946
#### NN---------------------: 0.9271
#### NN_xavier--------------: 0.9706
#### NN_xavier_Deep---------: 0.9653(과적합으로 정확도 감소)
#### NN_xavier_Deep_Dropout-: 0.9735

## Optimizer : 보통 Adam 많이쓰고 권장함(지금도 그런가?)
## Batch Normalization(일단 pass)

In [1]:
import tensorflow as tf_load_data
from keras.utils import np_utils
(x_train, y_train), (x_test, y_test) = tf_load_data.keras.datasets.mnist.load_data()

In [2]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# 예제용 reshape
x_train, x_test = x_train/255.0, x_test/255.0   #Feature scaling 적용, 그대록 학습시 문제됨...

# [[], [], ...] --> [...]
x_train_rs = x_train.reshape(x_train.shape[0], 784).astype('float32')
x_test_rs = x_test.reshape(x_test.shape[0], 784).astype('float32')

# one hot encoding
y_train_onehot = np_utils.to_categorical(y_train)
y_test_onehot = np_utils.to_categorical(y_test)

In [4]:
print(x_train_rs.shape)
print(y_train_onehot.shape)
print(y_test_onehot.shape)
print(x_test_rs.shape)

(60000, 784)
(60000, 10)
(10000, 10)
(10000, 784)


In [5]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import matplotlib.pyplot as plt
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
tf.set_random_seed(777)

In [7]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# SoftMax

In [8]:
nb_classes = 10

# MNIST data image of shape 28 * 28 = 784
X = tf.placeholder(tf.float32, [None, 784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, nb_classes])

W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

# Hypothesis (using softmax)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Test model
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [9]:
# parameters
num_epochs = 15
batch_size = 100
#num_iterations = int(mnist.train.num_examples / batch_size)
num_iterations = int(x_train.shape[0] / batch_size)

In [12]:
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(num_epochs):
        avg_cost = 0

        for i in range(num_iterations):
            #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            #_, cost_val = sess.run([train, cost], feed_dict={X: batch_xs, Y: batch_ys})
            batch_xs, batch_ys = next_batch(batch_size, x_train_rs, y_train_onehot)
            _, cost_val = sess.run([train, cost], feed_dict={X: batch_xs, Y: batch_ys})
            #print(cost_val)
            #print(num_iterations)
            avg_cost += cost_val / num_iterations

        print("Epoch: {:04d}, Cost: {:.9f}".format(epoch + 1, avg_cost))

    print("Learning finished")
    
    # Test the model using test sets
    print(
        "Accuracy: ",
        accuracy.eval(
            #session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}
            session=sess, feed_dict={X: x_test_rs, Y: y_test_onehot}
        ),
    )

Epoch: 0001, Cost: 2.833290844
Epoch: 0002, Cost: 1.097388055
Epoch: 0003, Cost: 0.863086824
Epoch: 0004, Cost: 0.739014846
Epoch: 0005, Cost: 0.673302667
Epoch: 0006, Cost: 0.622510433
Epoch: 0007, Cost: 0.587486926
Epoch: 0008, Cost: 0.554646429
Epoch: 0009, Cost: 0.529113251
Epoch: 0010, Cost: 0.518516070
Epoch: 0011, Cost: 0.480312823
Epoch: 0012, Cost: 0.473730461
Epoch: 0013, Cost: 0.475712282
Epoch: 0014, Cost: 0.463228252
Epoch: 0015, Cost: 0.444416175
Learning finished
Accuracy:  0.8946


# NN
### - Deep, relu 사용

In [14]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
print("Learning finished")
    
    # Test the model using test sets
    print(
        "Accuracy: ",
        accuracy.eval(
            #session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}
            session=sess, feed_dict={X: x_test_rs, Y: y_test_onehot}
        ),
    )

In [16]:
# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    #total_batch = int(mnist.train.num_examples / batch_size)
    total_batch = 100

    for i in range(total_batch):
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs, batch_ys = next_batch(batch_size, x_train_rs, y_train_onehot)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
print('Accuracy:', sess.run(accuracy, feed_dict={X: x_test_rs, Y: y_test_onehot}))

Epoch: 0001 cost = 23.240231164
Epoch: 0002 cost = 23.599039111
Epoch: 0003 cost = 20.681526769
Epoch: 0004 cost = 19.167079546
Epoch: 0005 cost = 19.179179697
Epoch: 0006 cost = 15.772219950
Epoch: 0007 cost = 16.080380872
Epoch: 0008 cost = 15.125584289
Epoch: 0009 cost = 15.648667257
Epoch: 0010 cost = 15.000378034
Epoch: 0011 cost = 14.346532525
Epoch: 0012 cost = 11.912314988
Epoch: 0013 cost = 12.110262533
Epoch: 0014 cost = 12.097198464
Epoch: 0015 cost = 10.934589062
Learning Finished!
Accuracy: 0.9271


# NN_xavier
### - Deep, relu 사용 + xavier방법으로 Weight 초기화

In [ ]:
# 테스트 버전에서는 "initializer=tf.contrib.layers.xavier_initializer())" 코드 사용불가(contrib 없는듯...)
# 아래 링크 참고로 유사한 기능에 코드로 대체
# https://stackoverflow.com/questions/59644859/attributeerror-module-tensorflow-core-compat-v1-has-no-attribute-contrib

In [19]:
W1 = tf.get_variable("W1", shape=[784, 256],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[256, 256],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[256, 10],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    #total_batch = int(mnist.train.num_examples / batch_size)
    total_batch = 100

    for i in range(total_batch):
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs, batch_ys = next_batch(batch_size, x_train_rs, y_train_onehot)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
print('Accuracy:', sess.run(accuracy, feed_dict={X: x_test_rs, Y: y_test_onehot}))

Epoch: 0001 cost = 0.638998076
Epoch: 0002 cost = 0.260522997
Epoch: 0003 cost = 0.203825704
Epoch: 0004 cost = 0.175977691
Epoch: 0005 cost = 0.130951064
Epoch: 0006 cost = 0.127342791
Epoch: 0007 cost = 0.109459504
Epoch: 0008 cost = 0.091833481
Epoch: 0009 cost = 0.092433945
Epoch: 0010 cost = 0.087961134
Epoch: 0011 cost = 0.079305443
Epoch: 0012 cost = 0.077013325
Epoch: 0013 cost = 0.068665789
Epoch: 0014 cost = 0.068031969
Epoch: 0015 cost = 0.056188521
Learning Finished!
Accuracy: 0.9706


# NN_xavier_Deep
### - Deep, relu 사용 + xavier방법으로 Weight 초기화
### + Deep

In [7]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [8]:
W1 = tf.get_variable("W1", shape=[784, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[512, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[512, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[512, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512, 10],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    #total_batch = int(mnist.train.num_examples / batch_size)
    total_batch = 100

    for i in range(total_batch):
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs, batch_ys = next_batch(batch_size, x_train_rs, y_train_onehot)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
print('Accuracy:', sess.run(accuracy, feed_dict={X: x_test_rs, Y: y_test_onehot}))

Epoch: 0001 cost = 1.280968966
Epoch: 0002 cost = 0.232107718
Epoch: 0003 cost = 0.180386321
Epoch: 0004 cost = 0.172244429
Epoch: 0005 cost = 0.127462742
Epoch: 0006 cost = 0.115443002
Epoch: 0007 cost = 0.099368928
Epoch: 0008 cost = 0.097800222
Epoch: 0009 cost = 0.081836102
Epoch: 0010 cost = 0.074979750
Epoch: 0011 cost = 0.077808304
Epoch: 0012 cost = 0.081205846
Epoch: 0013 cost = 0.067910093
Epoch: 0014 cost = 0.081668268
Epoch: 0015 cost = 0.065532887
Learning Finished!
Accuracy: 0.9653


# NN_xavier_Deep_Dropout
### - Deep, relu 사용 + xavier방법으로 Weight 초기화
### + Deep + Dropout

In [8]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
#total_batch = int(mnist.train.num_examples / batch_size)

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# dropout (keep_prob) rate  0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[784, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[512, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[512, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[512, 512],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[512, 10],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    #total_batch = int(mnist.train.num_examples / batch_size)
    total_batch = 100

    for i in range(total_batch):
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs, batch_ys = next_batch(batch_size, x_train_rs, y_train_onehot)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.8}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
print('Accuracy:', sess.run(accuracy, feed_dict={X: x_test_rs, Y: y_test_onehot, keep_prob: 1}))

Epoch: 0001 cost = 0.133428596
Epoch: 0002 cost = 0.111330585
Epoch: 0003 cost = 0.106143504
Epoch: 0004 cost = 0.108231284
Epoch: 0005 cost = 0.099682710
Epoch: 0006 cost = 0.094382100
Epoch: 0007 cost = 0.099571837
Epoch: 0008 cost = 0.082634827
Epoch: 0009 cost = 0.096847114
Epoch: 0010 cost = 0.093050286
Epoch: 0011 cost = 0.090213534
Epoch: 0012 cost = 0.089472233
Epoch: 0013 cost = 0.086094005
Epoch: 0014 cost = 0.081965483
Epoch: 0015 cost = 0.081699012
Learning Finished!
Accuracy: 0.9735
